# Verification Solveur 2D Calcul TRUST.

In [ ]:
from trustutils import run

run.introduction("T. Dumas, Y. Gorsse")
run.description("")
run.TRUST_parameters()

# Description des cas test

## Cas 1

Vitesse à direction oblique :

$V_x = V_y = e^{x-y}$

Domaine : 

           Entrée : x = -1 ; 0 <= y <= 2 et -1 <= x <= 1 ; y = 0

           Sortie : x = 1 ; 0 <= y <= 2 et -1 <= x <= 1 ; y = 2

## Cas 2

Vitesse à direction horizontale :

$V_x = 2 \sin(45°)e^{-2\cos(45°)x}$

$V_y = 0$

Domaine : 
           
           Entrée : x = -1 ; 0 <= y <= 2

           Sortie : x = 1 ; 0 <= y <= 2
           
           Paroi : -1 <= x <= 1 ; y = 0 et -1 <= x <= 1 ; y = 2
           
## Cas 3

Vitesse à direction horizontale :

$V_x = 2 \sin(45°)e^{-2\cos(45°)x}$

$V_y = 0$

Domaine : 

           Entrée : x = -1 ; 0 <= y <= 2 et -1 <= x <= 1 ; y = 0

           Sortie : x = 1 ; 0 <= y <= 2 et -1 <= x <= 1 ; y = 2

## Cas 4

Vitesse à direction oblique :

$V_x = [\cos(40°)+sin(40°)]e^{[(\cos(40°)-\sin(40°))x - (\cos(40°)+\sin(40°))y]}$

$V_y = [\cos(40°)-sin(40°)]e^{[(\cos(40°)-\sin(40°))x - (\cos(40°)+\sin(40°))y]}$

Domaine : 

           Entrée : x = -1 ; 0 <= y <= 2 et -1 <= x <= 1 ; y = 0

           Sortie : x = 1 ; 0 <= y <= 2 et -1 <= x <= 1 ; y = 2


NB : Tous les cas sont en condition initiale proche

In [ ]:
import os, sys
import copy
from string import Template


run.reset()
liste_maillage = ["11", "21", "41", "81"]
liste_dis = ["VDF", "PolyMAC", "CoviMAC"]
vxf = "(cos($theta)+sin($theta))*exp((cos($theta)-sin($theta))*x-(sin($theta)+cos($theta))*y)"
vyf = "(cos($theta)-sin($theta))*exp((cos($theta)-sin($theta))*x-(sin($theta)+cos($theta))*y)"
cases = {
    "cas1" : {"theta" :         "0", "cl_down" : "frontiere_ouverte_vitesse_imposee champ_front_fonc_txyz 2 exp(x-y) exp(x-y)", "cl_up" : "frontiere_ouverte_pression_imposee champ_front_uniforme 1 0"},
    "cas2" : {"theta" :      "pi/4", "cl_down" : "symetrie", "cl_up" : "symetrie"},
    "cas3" : {"theta" :      "pi/4", "cl_down" : "frontiere_ouverte_vitesse_imposee champ_front_fonc_txyz 2 2*sin(pi/4)*exp(-2*sin(pi/4)*y) 0", "cl_up" : "frontiere_ouverte_pression_imposee champ_front_uniforme 1 0"},
    "cas4" : {"theta" : "40*pi/180", "cl_down" : "frontiere_ouverte_vitesse_imposee champ_front_fonc_txyz 2 (cos(40*pi/180)+sin(40*pi/180))*exp((cos(40*pi/180)-sin(40*pi/180))*x-(sin(40*pi/180)+cos(40*pi/180))*y) (cos(40*pi/180)-sin(40*pi/180))*exp((cos(40*pi/180)-sin(40*pi/180))*x-(sin(40*pi/180)+cos(40))*y)", "cl_up" : "frontiere_ouverte_pression_imposee champ_front_uniforme 1 0"},
}

for dis in liste_dis:
    for case, data in cases.items():
        for n in liste_maillage:
            d = copy.deepcopy(data)
            d["nx"] = n
            d["ny"] = n
            d["dis"] = dis
            run.addCaseFromTemplate("jdd.data", f"{dis}/{case}/maillage_{n}", d)

run.runCases()
run.tablePerf()

## Vérification Convergence

In [ ]:
from trustutils import plot
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.colors as mcolors
c = list(mcolors.TABLEAU_COLORS.keys())

def func(x, a, b): return a * x ** b

length = 2.0
dz = [length / ((float(x) - 1.0)) for x in liste_maillage]
dirs = ["X", "Y"]
norms = ["L2", "LMAX"]
linestyle = {"PolyMAC": ("solid", "o"), "VDF" : ("dotted", "*"), "CoviMAC" : ("dashed", "v")}

for case, data in cases.items():
    error = {dis : {norm : {d: [] for d in dirs} for norm in norms} for dis in liste_dis}
    for n in liste_maillage:
        for norm in norms:
            for d in dirs:
                for dis in liste_dis:
                    x = plot.loadText(f"{dis}/{case}/maillage_{n}/jdd_{norm}_{d}.son")
                    error[dis][norm][d].append(x[-1][-1])
    Graph = plot.Graph(f"Vérification de Convergence {case}")
    for k, (dis, norm_data) in enumerate(error.items()):
        for norm, dirs_data in norm_data.items():
            for d, err in dirs_data.items():
                (_, b), _ = curve_fit(func, dz, err)
                Graph.add(dz, err, label=f"{dis} {norm}_{d}, order={b:.2f}", color=c[k], linestyle=linestyle[dis][0], marker=linestyle[dis][1], markersize=12)
    Graph.scale(xscale='log',yscale='log')
    Graph.label(r"$aNe^{-1/2}$","erreur")